# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_Booth import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
df = pd.read_csv("my_data.csv")
df.head()

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
0,Elk City,US,35.4120,-99.4043,1.644076e+09,9.22,1030.0,59.0,26.92,0.0
1,Price,US,39.5994,-110.8107,1.644076e+09,3.44,1028.0,73.0,16.54,0.0
2,Georgiyevskoye,RU,58.7310,45.0240,1.644076e+09,5.84,1012.0,95.0,11.73,77.0
3,Plettenberg Bay,ZA,-34.0527,23.3716,1.644076e+09,23.44,1016.0,76.0,72.48,2.0
4,umzimvubu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_sub = df.dropna().reset_index(drop=True)
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         55 non-null     object 
 1   Country      55 non-null     object 
 2   Latitude     55 non-null     float64
 3   Longitude    55 non-null     float64
 4   Date         55 non-null     float64
 5   Wind_Speed   55 non-null     float64
 6   Pressure     55 non-null     float64
 7   Humidity     55 non-null     float64
 8   Temperature  55 non-null     float64
 9   Cloudiness   55 non-null     float64
dtypes: float64(8), object(2)
memory usage: 4.4+ KB


In [15]:
df_sub.describe()

,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
count,55.000000,55.000000,5.500000e+01,55.000000,55.000000,55.000000,55.000000,55.000000
mean,14.332105,-1.962249,1.644076e+09,8.468727,1015.363636,76.400000,46.453091,48.527273
std,39.797523,102.802710,8.927400e+01,7.470146,11.952628,16.776968,34.000188,40.932127
min,-54.800000,-175.200000,1.644075e+09,0.000000,965.000000,41.000000,-40.320000,0.000000
25%,-24.435150,-81.871100,1.644076e+09,3.430000,1010.500000,66.500000,26.875000,2.000000
50%,22.545500,-9.266700,1.644076e+09,6.260000,1016.000000,77.000000,57.970000,41.000000
75%,41.734100,94.366300,1.644076e+09,9.820000,1022.000000,91.500000,72.600000,96.500000
max,72.786800,169.846400,1.644076e+09,31.090000,1037.000000,100.000000,86.400000,100.000000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [33]:
locations = df_sub.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df_sub['Humidity'],
    max_intensity=20, point_radius=5
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# A max temperature lower than 80 degrees but higher than 70.


# Wind speed less than 10 mph.


# Zero cloudiness

mask = (df_sub.Temperature < 80) & (df_sub.Temperature > 70) & (df_sub.Wind_Speed < 10) & (df.Cloudiness == 0)

good_cities = df_sub.loc[mask]
good_cities

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
13,Kahului,US,20.8947,-156.4700,1.644076e+09,4.61,1017.0,82.0,70.25,0.0
40,Key Largo,US,25.0865,-80.4473,1.644076e+09,6.55,1022.0,74.0,77.13,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
# geocoordinates
target_coordinates = f"{20.8947}, {-156.4700}"
target_search = ""

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

places_data = response.json()

print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])
print(places_data["results"][0]["rating"])

Maui Seaside Hotel
100 West Kaahumanu Avenue, Kahului
3.8


In [25]:
hotel_names = []
ratings = []
addresses = []
cities = []
countries = []
lats = []
longs = []

for _, row in good_cities.iterrows():
    print(row.City)

    # geocoordinates
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = ""

    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()

    hotel = places_data["results"][0]["name"]
    address = places_data["results"][0]["vicinity"]
    rating = places_data["results"][0]["rating"]
    
    # put it all in a list
    hotel_names.append(hotel)
    ratings.append(rating)
    addresses.append(address)
    cities.append(row.City)
    countries.append(row.Country)
    lats.append(row.Latitude)
    longs.append(row.Longitude)

Kahului
Key Largo


In [26]:
hotel_df = pd.DataFrame()
hotel_df['City'] = cities
hotel_df['Country'] = countries
hotel_df['Latitude'] = lats
hotel_df['Longitude'] = longs
hotel_df['Hotel'] = hotel_names
hotel_df['Address'] = addresses
hotel_df['Rating'] = ratings

hotel_df

,City,Country,Latitude,Longitude,Hotel,Address,Rating
0,Kahului,US,20.8947,-156.4700,Maui Seaside Hotel,"100 West Kaahumanu Avenue, Kahului",3.8
1,Key Largo,US,25.0865,-80.4473,Bayside Inn Key Largo,"99490 Overseas Highway, Key Largo",3.6


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [34]:
# create layers
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df_sub['Humidity'],
    max_intensity=20, point_radius=5
)


fig = gmaps.figure()

# add layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)


fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
